# **Inpaint stable diffusion**

Inpaint notebook to create a new image from a mask image and a prompt

## Check Environment

In [1]:
import os

if 'COLAB_GPU' in os.environ:
    print("Environment is colab")
    env = "colab"
elif 'KAGGLE_URL_BASE' in os.environ:
    env = "kaggle"
    print("Environment is kaggle")
else:
    env = "local"
    print("Environment is local")

Environment is local


### Set ENV_TYPE
If ENV_TYPE is equal to "test", then some examples of data will be run in order to test whether the notebooks running or not.
Otherwise, the notebook will run on the whole dataset

In [2]:
ENV_TYPE="TEST"

model_path = "runwayml/stable-diffusion-inpainting"

# **Install the requirements**

In [1]:
!pip install -qq -U diffusers==0.11.1 transformers ftfy gradio accelerate

# **Computing Platform Check GPU (CUDA) or CPU / Environment**


In [2]:
import torch
if torch.cuda.is_available():
    device = "cuda"
else:
    print ('[WARNING] CUDA/GPU is not available! Compute-intensive scripts on this notebook will be run on CPU.')
    device =  "cpu"

# **Import the Module/Utility**

In [3]:
import inspect
from typing import List, Optional, Union
import random
import numpy as np
import torch

import requests
from io import BytesIO
from PIL import Image
import gradio as gr
from diffusers import StableDiffusionInpaintPipeline

### **Create Inpaint pipeline**

In [4]:
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
).to(device)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


### **Create Inpaint funtion**

In [5]:
def inpaint(ori_image, mask_image, prompt, guidance_scale=7.5, num_samples=3, seed=0):
    ori_image = Image.open(ori_image)
    mask_image = Image.open(mask_image)
    generator = torch.Generator(device="cuda").manual_seed(seed)
    image =  ori_image.convert("RGB").resize((512, 512))
    mask_image = mask_image.convert("RGB").resize((512, 512))
    images = pipe(prompt=prompt, 
                  image=image, 
                  mask_image=mask_image, 
                  guidance_scale=guidance_scale,
                  generator=generator,
                  num_images_per_prompt=num_samples).images
    return(images)

### **Set parameters**

In [6]:
def download_image(url):
    response = requests.get(url)
    return BytesIO(response.content)

In [7]:
img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
img_url = download_image(img_url)
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"
mask_url = download_image(mask_url)
prompt = "a mecha robot sitting on a bench"
guidance_scale=7.5
num_samples = 3

### **Generate inpaint images**

In [8]:
images = inpaint(img_url, mask_url, prompt, num_samples=num_samples)

  0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.50 GiB (GPU 0; 5.80 GiB total capacity; 4.20 GiB already allocated; 376.69 MiB free; 4.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### **Show images**

In [ ]:
import math
def image_grid(imgs, rows=None, cols=None):
    if not cols:
        if len(imgs) > 5:
            cols = 5
        else:
            cols = len(imgs)
        rows = math.ceil(len(imgs)/cols)
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

Show original and mask image

In [ ]:
image = Image.open(img_url)
image

In [ ]:
image = Image.open(mask_url)
image

Show inpaint generated images

In [ ]:
image_grid(images)

### **Manual upload input image and create mask image**

In [ ]:
def predict(dict, prompt):
  image =  dict['image'].convert("RGB").resize((512, 512))
  mask_image = dict['mask'].convert("RGB").resize((512, 512))
  images = pipe(prompt=prompt, image=image, mask_image=mask_image).images
  return(images[0])

In [ ]:
gr.Interface(
    predict,
    title = 'Stable Diffusion In-Painting',
    inputs=[
        gr.Image(source = 'upload', tool = 'sketch', type = 'pil'),
        gr.Textbox(label = 'prompt')
    ],
    outputs = [
        gr.Image()
        ]
).launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>